In [ ]:
!pip install  torchxrayvision

import torchxrayvision

In [ ]:
from pyparsing.helpers import PositionToken
from pandas import Index
#Code is first test on https://github.com/mlmed/torchxrayvision
import torchxrayvision as xrv
import skimage, torch, torchvision
import matplotlib.pyplot as plt
import os
import tabulate
import cv2 as cv

#added code
from google.colab import drive
from PIL import Image
from os import listdir
from tabulate import tabulate
drive.mount('/content/drive')
#end of added code

path = "/content/drive/MyDrive/Shared Capstone/Colab/starting_images"

file_name = "00000001_001.png"

# Prepare the image:
original_img = Image.open(path + "/" + file_name)
modified_img = original_img.convert('RGB')
modified_img.save('image.jpg')

#read the image
image = cv.imread("image.jpg")
print(image.shape)

img = skimage.io.imread('image.jpg')
plt.imshow(img);
img = xrv.datasets.normalize(img, 255) # convert 8-bit image to [-1024, 1024] range
img = img.mean(2)[None, ...] # Make single color channel

transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),xrv.datasets.XRayResizer(224)])
img = transform(img)
img = torch.from_numpy(img)

# Load model and process image

model = xrv.models.DenseNet(weights="densenet121-res224-nih") # NIH chest X-ray8
outputs = model(img[None,...]) # or outputs = model.features(img[None,...])
probability_dictionary = dict(zip(model.pathologies,outputs[0].detach().numpy()))

line_content = []

with open("/content/drive/MyDrive/Shared Capstone/Colab/Data_Entry_2017_Unique.csv") as myFile:
    for num, line in enumerate(myFile, 0):
        if file_name in line:
            line_content = line.split(",")
            print("the line content is: ", line_content)

patient_data_dict = {
  "Image Index": line_content[0],
  "Finding Labels": line_content[1],
  "Follow Up Number": line_content[2],
  "Patient ID": line_content[3],
  "Patient Age": line_content[4],
  "Patient Gender": line_content[5],
  "View Position": line_content[6],
  "Original Image Width": line_content[7],
  "Original Image Height": line_content[8],
  "Original Image Pixel Spacing x": line_content[9],
  "Original Image Pixel Spacing y": line_content[10]
}

print("\n")

print("Patient & Image Data: \n")

print(tabulate((patient_data_dict.items())))

actual_dictionary = {
    'Atelectasis': 0,
    'Cardiomegaly': 0,
    'Consolidation': 0,
    'Edema': 0,
    'Effusion': 0
}

diseases = line_content[1].split("|")

if ('Atelectasis' in diseases): actual_dictionary['Atelectasis'] = 1
if ('Cardiomegaly' in diseases): actual_dictionary['Cardiomegaly'] = 1
if ('Consolidation' in diseases): actual_dictionary['Consolidation'] = 1
if ('Edema' in diseases): actual_dictionary['Edema'] = 1
if ('Effusion' in diseases): actual_dictionary['Effusion'] = 1

array1 = ['Atelectasis', probability_dictionary['Atelectasis'], actual_dictionary['Atelectasis']]
array2 = ['Cardiomegaly', probability_dictionary['Cardiomegaly'], actual_dictionary['Cardiomegaly']]
array3 = ['Consolidation', probability_dictionary['Consolidation'], actual_dictionary['Consolidation']]
array4 = ['Edema', probability_dictionary['Edema'], actual_dictionary['Edema']]
array5 = ['Effusion', probability_dictionary['Effusion'], actual_dictionary['Effusion']]

table = [['Disease', 'Probability', 'Actual'], array1, array2, array3, array4, array5]

print("\n")
print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))
